In [1]:
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer,WordNetLemmatizer
import pandas as pd
import re
from textblob import TextBlob
import emoji
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy as s
import scipy.stats as ss
import warnings 
warnings.filterwarnings('ignore') 
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
pip install langcodes

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install translate

Note: you may need to restart the kernel to use updated packages.


# Insights of Dataset


text given in each datapoint is a tweet or grabage values with is in many languages. the data is given in 4 files with label as the file name. as machine can be trained by only numerical values , we have to convert it to vectors ,before that all the text should be in same language , so we can translate the text from non-english to english and train the machine or by dropping tweets which are in non-english language

In [2]:
from translate import Translator

In [3]:
d1=pd.read_csv("C:\\Users\\Praveen kumar\\Downloads\\dataset\\positive.csv",encoding="latin-1")
d2=pd.read_csv("C:\\Users\\Praveen kumar\\Downloads\\dataset\\uncertainty.csv",encoding="latin-1")
d3=pd.read_csv("C:\\Users\\Praveen kumar\\Downloads\\dataset\\litigious.csv",encoding="latin-1")
d4=pd.read_csv("C:\\Users\\Praveen kumar\\Downloads\\dataset\\negative.csv",encoding="latin-1")

concatinating all the 4 dataframes in to one

In [4]:
data=pd.concat([d1,d2,d3,d4])
data

,Unnamed: 0,Text,Language
0,3,Rwanda is set to host the headquarters of Unit...,en
1,12,Wimbledon: Nick Kyrgios admits spitting toward...,en
2,13,is booktwt a thing? if so thats her and she sp...,en
3,18,At the end we consume fictional stories to enj...,en
4,19,#Duranto Express A1 coach AC not working prope...,en
...,...,...,...
262215,937846,@SBeaberry @EmmaVigeland No I haven't because ...,en
262216,937847,Yâall talking bad on that dad feeding his on...,en
262217,937850,8.SSR &amp; Disha Salian case should be solved...,en
262218,937851,*ACCIDENT: Damage Only* - Raleigh Fire Depart...,en


In [5]:
data.sort_values('Unnamed: 0',inplace=True)

reseting all the index values as we have concatinated many dataframes to one

In [6]:
data = data.reset_index()

In [7]:
lang = data['Language'].unique().tolist()
lang

['en',
 'es',
 'ca',
 'fr',
 'pt',
 'ja',
 'hi',
 'ar',
 'ko',
 'nl',
 'et',
 'in',
 'und',
 'tl',
 'zh',
 'de',
 'cs',
 'ru',
 'qme',
 'it',
 'el',
 'ro',
 'no',
 'tr',
 'sv',
 'ta',
 'fa',
 'ht',
 'pl',
 'da',
 'th',
 'hu',
 'lv',
 'uk',
 'qht',
 'eu',
 'qam',
 'si',
 'cy',
 'zxx',
 'ml',
 'ne',
 'mr',
 'qst',
 'vi',
 'bn',
 'gu',
 'is',
 'fi',
 'ckb',
 nan,
 'te',
 'art',
 'bg',
 'ur',
 'sl',
 'lt',
 'pa',
 'iw',
 'kn',
 'sr',
 "[Photo(previewUrl='https://pbs.twimg.com/media/FWV94O7UEAAjMOi?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FWV94O7UEAAjMOi?format=jpg&name=large')]",
 "[Photo(previewUrl='https://pbs.twimg.com/media/FWWBzWTXkAAyZqm?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FWWBzWTXkAAyZqm?format=jpg&name=large')]",
 'am',
 'or',
 "[Photo(previewUrl='https://pbs.twimg.com/media/EPYG2rKVAAA1e_O?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/EPYG2rKVAAA1e_O?format=jpg&name=large')]",
 'sd',
 "[Photo(previewUrl='https://pbs.twimg

In [7]:
data.dropna(inplace=True)

In [8]:
data

,index,Unnamed: 0,Text,Language
0,0,0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en
1,0,1,#BadBunny: Como dos gotas de agua: Joven se di...,es
2,1,2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en
3,0,3,Rwanda is set to host the headquarters of Unit...,en
4,2,4,OOPS. I typed her name incorrectly (todayâs ...,en
...,...,...,...,...
937849,264543,937849,@Juice_Lemons in the dark. itâs so good,en
937850,262217,937850,8.SSR &amp; Disha Salian case should be solved...,en
937851,262218,937851,*ACCIDENT: Damage Only* - Raleigh Fire Depart...,en
937852,264544,937852,@reblavoie So happy for her! Sheâs been incr...,en


Now all the tweets are which we took is in english language so that we can transform to vectors easily

In [9]:
data = data[data["Language"]=="en"]
data

,index,Unnamed: 0,Text,Language
0,0,0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en
2,1,2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en
3,0,3,Rwanda is set to host the headquarters of Unit...,en
4,2,4,OOPS. I typed her name incorrectly (todayâs ...,en
5,1,5,It sucks for me since I'm focused on the natur...,en
...,...,...,...,...
937849,264543,937849,@Juice_Lemons in the dark. itâs so good,en
937850,262217,937850,8.SSR &amp; Disha Salian case should be solved...,en
937851,262218,937851,*ACCIDENT: Damage Only* - Raleigh Fire Depart...,en
937852,264544,937852,@reblavoie So happy for her! Sheâs been incr...,en


In [10]:
stop_words = stopwords.words("english")
stop_words.remove('not')

this stop words which occur often and meaning of sentance will not be changed if they are not present , so that dimensions of the vector can be reduced , not is removed assuming that if that is removed from the sentance the meaning could be reversed

In [ ]:
%%time
sentances = []
for i in data['Text']:
    word = []
    sent = i.lower()
    sent = emoji.demojize(sent)
    sent = re.sub('â€˜|â€™',"'",sent)
    sent = re.sub('&amp;',"and",sent)
    sent = re.sub('<.*?>','',sent)
    sent = re.sub('https?://\S*|www\S*','',sent)
    sent = re.sub(' @\w+| #\w+',"",sent)
    sent = re.sub("[^a-z0-9 ]","",sent)
    for i in word_tokenize(sent):
        if i in stop_words:pass
        else:word.append(PorterStemmer().stem(i))
    sentances.append(" ".join(word))

In the above step ive done the text pre-processing , this all the steps inside are done for redcution of vector dimensions

In [14]:
sentances

['charliecorley testimoni not evid court law state feder must stand cross examin',
 'flagstar bank disclos data breach impact 15million individu',
 'rwanda set host headquart unit nation develop programm undp new innov financ facil find sovereigngroup',
 'oop type name incorrectli today brave wit 6 7 probabi dont forget request',
 'suck sinc im focus natur aspect thing environment problem root social econom polit issu simpli recycl use renew energi like fix top floor build unsteadi foundat',
 'shawntarloff also relat art lot peopl dismay start art kind thing alway tri tell everyon start somewher good certain age train eno',
 'social secur constant polit crise distract mass rob us blind meantim come american peopl breathtak speed everi direct til peopl dont know look first not accid',
 'filmthepolicela broken rib punctur lung lead collaps lung much longer recoveri break get check',
 'jacobringenwald agre flaw respons small mani pl peopl presum would agre nuanc understand need casesbut g

This is the final output of a dataFrame after doing text pre-processing

In [17]:
cd = pd.DataFrame({"c_text":sentances})
cd

,c_text
0,charliecorley testimoni not evid court law sta...
1,flagstar bank disclos data breach impact 15mil...
2,rwanda set host headquart unit nation develop ...
3,oop type name incorrectli today brave wit 6 7 ...
4,suck sinc im focus natur aspect thing environm...
...,...
871305,juicelemon dark good
871306,8ssr disha salian case solv drug cartel finsh ...
871307,accid damag raleigh fire depart 2503 fairway dr
871308,reblavoi happi she incred


In [29]:
#bag of words
cv = CountVectorizer()
t = cv.fit_transform(cd['c_text'][0:50000])
t

<50000x75099 sparse matrix of type '<class 'numpy.int64'>'
	with 642092 stored elements in Compressed Sparse Row format>

There are total of 642092 unique words in the dataset

In [30]:
count_array = t.toarray()
count_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
cv.vocabulary_

{'charliecorley': 13814,
 'testimoni': 65713,
 'not': 47607,
 'evid': 23382,
 'court': 16635,
 'law': 38656,
 'state': 63085,
 'feder': 24385,
 'must': 45629,
 'stand': 62963,
 'cross': 17105,
 'examin': 23469,
 'flagstar': 25054,
 'bank': 8398,
 'disclos': 19864,
 'data': 18110,
 'breach': 11152,
 'impact': 32322,
 '15million': 625,
 'individu': 32693,
 'rwanda': 57749,
 'set': 59770,
 'host': 31079,
 'headquart': 29778,
 'unit': 69841,
 'nation': 46128,
 'develop': 19334,
 'programm': 53422,
 'undp': 69718,
 'new': 46641,
 'innov': 32985,
 'financ': 24787,
 'facil': 23881,
 'find': 24799,
 'sovereigngroup': 62315,
 'oop': 49102,
 'type': 69195,
 'name': 45925,
 'incorrectli': 32569,
 'today': 67566,
 'brave': 11114,
 'wit': 73000,
 'probabi': 53226,
 'dont': 20502,
 'forget': 25557,
 'request': 56026,
 'suck': 63995,
 'sinc': 60944,
 'im': 32218,
 'focus': 25346,
 'natur': 46166,
 'aspect': 7117,
 'thing': 66719,
 'environment': 22733,
 'problem': 53245,
 'root': 57276,
 'social': 61

This is the list of all the unique words in DataSet with their index values in the form of dictionary

In [32]:
pd.DataFrame(count_array,columns=sorted(cv.vocabulary_))

,00,000,0000,0000238,0001,000tacitu,002,004,00447935431747,0047,...,zydu,zygot,zygotes3,zyiech,zylstra,zysnag,zzh,zzztyler,zzzzz,zzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#TF-IDF
tfv = TfidfVectorizer()
trans_tf = tfv.fit_transform(cd['c_text'][0:15000])
trans_tf

<15000x31129 sparse matrix of type '<class 'numpy.float64'>'
	with 192454 stored elements in Compressed Sparse Row format>

In [45]:
trans_tf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
tfv.vocabulary_

{'charliecorley': 13814,
 'testimoni': 65713,
 'not': 47607,
 'evid': 23382,
 'court': 16635,
 'law': 38656,
 'state': 63085,
 'feder': 24385,
 'must': 45629,
 'stand': 62963,
 'cross': 17105,
 'examin': 23469,
 'flagstar': 25054,
 'bank': 8398,
 'disclos': 19864,
 'data': 18110,
 'breach': 11152,
 'impact': 32322,
 '15million': 625,
 'individu': 32693,
 'rwanda': 57749,
 'set': 59770,
 'host': 31079,
 'headquart': 29778,
 'unit': 69841,
 'nation': 46128,
 'develop': 19334,
 'programm': 53422,
 'undp': 69718,
 'new': 46641,
 'innov': 32985,
 'financ': 24787,
 'facil': 23881,
 'find': 24799,
 'sovereigngroup': 62315,
 'oop': 49102,
 'type': 69195,
 'name': 45925,
 'incorrectli': 32569,
 'today': 67566,
 'brave': 11114,
 'wit': 73000,
 'probabi': 53226,
 'dont': 20502,
 'forget': 25557,
 'request': 56026,
 'suck': 63995,
 'sinc': 60944,
 'im': 32218,
 'focus': 25346,
 'natur': 46166,
 'aspect': 7117,
 'thing': 66719,
 'environment': 22733,
 'problem': 53245,
 'root': 57276,
 'social': 61

In [46]:
pd.DataFrame(trans_tf.toarray(),columns=sorted(tfv.vocabulary_.keys()))

,00,000,0001,004,00447935431747,0047,007,01,02,020,...,zuelthecat,zuk,zulfiqarahmed69,zumakoko,zxclark,zychaxrma,zygot,zyiech,zylstra,zzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


in countvectorizer (bag of words) and tfidfvectorizer(TF-IDF) i have reduced datapoints because of memory and time issue so i have limited to 50000 rows in countvectorizer instead of 871310 and 15000 in tfidfvectorizer
